In [12]:
import pandas as pd
import numpy as np

In [13]:
data = pd.read_csv("dataframe_ADE.csv")
data.drop(["Unnamed: 0"], axis=1, inplace=True)

In [14]:
data = data.fillna(method="ffill")

In [15]:
data.head(10)

,Sentence_ID,Token_ID,BIO,POS,Token,ID
0,Sentence: 1,1,O,JJ,Gemcitabine-related,0
1,Sentence: 1,2,B_AdverseReaction,NN,radiation,1
2,Sentence: 1,3,I_AdverseReaction,NN,recall,2
3,Sentence: 1,4,O,IN,in,3
4,Sentence: 1,5,O,DT,a,4
5,Sentence: 1,6,O,NN,patient,5
6,Sentence: 1,7,O,IN,with,6
7,Sentence: 1,8,O,JJ,pancreatic,7
8,Sentence: 1,9,O,NN,cancer,8
9,Sentence: 1,10,O,.,.,9


In [11]:
data.groupby('BIO').count()

,Sentence_ID,Token_ID,POS,Token,ID
BIO,,,,,
B_AdverseReaction,5749,5749,5749,5749,5749
I_AdverseReaction,6529,6529,6529,6529,6529
O,73832,73832,73832,73832,73832


In [5]:
words = list(set(data["Token"].values))

In [6]:
n_words = len(words); n_words

9211

In [8]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Token"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["BIO"].values.tolist())]
        self.grouped = self.data.groupby("Sentence_ID").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [9]:
getter = SentenceGetter(data)

In [10]:
sent = getter.get_next()

In [11]:
print(sent)

[('Gemcitabine-related', 'JJ', 'O'), ('radiation', 'NN', 'B_AdverseReaction'), ('recall', 'NN', 'I_AdverseReaction'), ('in', 'IN', 'O'), ('a', 'DT', 'O'), ('patient', 'NN', 'O'), ('with', 'IN', 'O'), ('pancreatic', 'JJ', 'O'), ('cancer', 'NN', 'O'), ('.', '.', 'O')]


In [12]:
sentences = getter.sentences

In [13]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [14]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [15]:
from sklearn_crfsuite import CRF

crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [16]:
from sklearn.cross_validation import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report

/Users/symeonidoua/.local/share/virtualenvs/nerds-b4u6_Kwr/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [17]:
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

In [18]:
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

                   precision    recall  f1-score   support

B_AdverseReaction       0.79      0.73      0.76      5749
I_AdverseReaction       0.78      0.71      0.74      6529
                O       0.96      0.97      0.97     73832

      avg / total       0.94      0.94      0.94     86110



In [19]:
crf.fit(X, y)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=False, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [20]:
crf = CRF(algorithm='lbfgs',
          c1=10,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [21]:
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

In [22]:
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

                   precision    recall  f1-score   support

B_AdverseReaction       0.72      0.48      0.58      5749
I_AdverseReaction       0.72      0.50      0.59      6529
                O       0.93      0.97      0.95     73832

      avg / total       0.90      0.91      0.90     86110



In [23]:
crf.fit(X, y)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=False, averaging=None, c=None, c1=10, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)